## 전이학습-2 <br>
## 분류 3개로 (Q1+Q2 / Q3+Q4 / Q5) <br>
## 런타임마다 재실행해야 할 목록

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob, cv2
from PIL import Image
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
import keras
import tensorflow as tf
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [3]:
#path 정리하기
# model_path = "/content/drive/MyDrive/2.양추정/음식 분류 모델/model_j.hdf5"

# x = np.load('/content/drive/MyDrive/2.양추정/np.array_1_y값 변경전/X_data.npy')     #기존 버전 나중에 주석처리하기
X = np.load('/content/drive/MyDrive/2.양추정/np.array_3_전체X값_255/X_data_float16.npy')
y = np.load('/content/drive/MyDrive/2.양추정/np.array_1_y값 변경전/y_data.npy')

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (40754, 224, 224, 3)
y shape: (40754,)


In [ ]:
#이방법 노노 모델이 너무 큼
# c = x/255.0
# X_type = c.astype(np.float32)

# np.save('/content/drive/MyDrive/2.양추정/np.array_3_전체X값_255/X_data_all.npy', X_type)

In [ ]:
# #모델을 줄이자
# c = x/255.0
# X_type = c.astype(np.float16)

# np.save('/content/drive/MyDrive/2.양추정/np.array_3_전체X값_255/X_data_float16.npy', X_type)

In [ ]:
# X = np.load('/content/drive/MyDrive/2.양추정/np.array_3_전체X값_255/X_data_all.npy')
# print("X_all shape:", X.shape)

In [ ]:
# plt.imshow(X[0])
# plt.title('Label: ' + str(y[0]))
# plt.axis('off') # 축 제거
# plt.show()

In [ ]:
#라벨(1~5) 확인하기
a = np.unique(y)
a

array([1, 2, 3, 4, 5])

In [4]:
#y값 라벨링 다시 하기
#2를 1로 / 4를 3으로

y_label = np.where(y == 2, 1, np.where(y == 4, 3, y))
y_label

array([1, 1, 1, ..., 5, 5, 5])

In [5]:
#라벨(1,3,5)) 확인하기
b = np.unique(y_label)
b

array([1, 3, 5])

In [6]:
#원-핫 인코딩
Y = pd.get_dummies(y_label)
Y

,1,3,5
0,True,False,False
1,True,False,False
2,True,False,False
3,True,False,False
4,True,False,False
...,...,...,...
40749,False,False,True
40750,False,False,True
40751,False,False,True
40752,False,False,True


In [7]:
#split
#전체: 40754
#train 데이터 크기: (32603, 224, 224, 3) (32603, 3)
#test 데이터 크기: (8151, 224, 224, 3) (8151, 3)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=y, random_state=42)

print("train 데이터 크기:", X_train.shape, y_train.shape)
print("test 데이터 크기:", X_test.shape, y_test.shape)

train 데이터 크기: (32603, 224, 224, 3) (32603, 3)
test 데이터 크기: (8151, 224, 224, 3) (8151, 3)




---

## 모델 학습

In [8]:
#model8 - 1
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

yang_model = Sequential()
yang_model.add(base_model)
yang_model.add(GlobalAveragePooling2D())
yang_model.add(Dropout(0.2))
yang_model.add(Dense(32, activation='relu'))
yang_model.add(Dropout(0.2))
yang_model.add(Dense(3, activation='softmax'))

# 새로운 모델 컴파일
yang_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 새로운 모델 요약
yang_model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model8-1.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

history = yang_model.fit(X_train, y_train, validation_split=0.25, epochs=100, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

score = yang_model.evaluate(X_test, y_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

234545216/234545216 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 7, 7, 2048)        58331648  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 32)                65568     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


583/583 [==============================] - 3535s 6s/step - loss: 0.9613 - accuracy: 0.5191 - val_loss: 20.5906 - val_accuracy: 0.3685
Epoch 2/100
583/583 [==============================] - ETA: 0s - loss: 0.8566 - accuracy: 0.5840
Epoch 2: val_loss improved from 20.59058 to 0.96194, saving model to /content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model8-1.h5
583/583 [==============================] - 3447s 6s/step - loss: 0.8566 - accuracy: 0.5840 - val_loss: 0.9619 - val_accuracy: 0.5171
Epoch 3/100
583/583 [==============================] - ETA: 0s - loss: 0.7434 - accuracy: 0.6504
Epoch 3: val_loss did not improve from 0.96194
583/583 [==============================] - 3414s 6s/step - loss: 0.7434 - accuracy: 0.6504 - val_loss: 1.1064 - val_accuracy: 0.5046
Epoch 4/100
583/583 [==============================] - ETA: 0s - loss: 0.6538 - accuracy: 0.7028
Epoch 4: val_loss improved from 0.96194 to 0.83156, saving model to /content/drive/MyDrive/YANG/model/checkpoint/model_

KeyboardInterrupt: 

In [9]:
#model8 - 2
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

yang_model = Sequential()
yang_model.add(base_model)
yang_model.add(Dense(32, activation='relu'))
yang_model.add(Dropout(0.25))
yang_model.add(Flatten())
yang_model.add(Dense(3, activation='softmax'))

# 새로운 모델 컴파일
yang_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 새로운 모델 요약
yang_model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model8-2.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

history = yang_model.fit(X_train, y_train, validation_split=0.25, epochs=100, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

score = yang_model.evaluate(X_test, y_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet152v2 (Functional)    (None, 7, 7, 2048)        58331648  
                                                                 
 dense_2 (Dense)             (None, 7, 7, 32)          65568     
                                                                 
 dropout_2 (Dropout)         (None, 7, 7, 32)          0         
                                                                 
 flatten (Flatten)           (None, 1568)              0         
                                                                 
 dense_3 (Dense)             (None, 3)                 4707      
                                                                 
Total params: 58401923 (222.79 MB)
Trainable params: 58258179 (222.24 MB)
Non-trainable params: 143744 (561.50 KB)
_________________________________________________________________
Epoch

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


583/583 [==============================] - 3491s 6s/step - loss: 1.0749 - accuracy: 0.4025 - val_loss: 1.0608 - val_accuracy: 0.4045
Epoch 2/100
583/583 [==============================] - ETA: 0s - loss: 1.0512 - accuracy: 0.4168
Epoch 2: val_loss did not improve from 1.06082
583/583 [==============================] - 3361s 6s/step - loss: 1.0512 - accuracy: 0.4168 - val_loss: 1.0834 - val_accuracy: 0.3857
Epoch 3/100
583/583 [==============================] - ETA: 0s - loss: 1.0057 - accuracy: 0.4749
Epoch 3: val_loss improved from 1.06082 to 1.03243, saving model to /content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model8-2.h5
583/583 [==============================] - 3414s 6s/step - loss: 1.0057 - accuracy: 0.4749 - val_loss: 1.0324 - val_accuracy: 0.4434
Epoch 4/100
583/583 [==============================] - ETA: 0s - loss: 0.9835 - accuracy: 0.4907
Epoch 4: val_loss did not improve from 1.03243
583/583 [==============================] - 3446s 6s/step - loss: 0.9835 - 

KeyboardInterrupt: 

In [ ]:
#model8 - 3
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

yang_model = Sequential()
yang_model.add(base_model)
yang_model.add(Dense(64, activation='relu'))
yang_model.add(Dropout(0.25))
yang_model.add(Flatten())
yang_model.add(Dense(3, activation='softmax'))

# 새로운 모델 컴파일
yang_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 새로운 모델 요약
yang_model.summary()

checkpoint_path = "/content/drive/MyDrive/YANG/model/checkpoint/model_checkpoint_model8-3.h5"
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

history = yang_model.fit(X_train, y_train, validation_split=0.25, epochs=50, batch_size=42, verbose=1, callbacks=[checkpoint, early_stopping])

score = yang_model.evaluate(X_test, y_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])